<br>
<center>
<img src="https://raw.githubusercontent.com/elaynelemos/prediction-of-orders-dmc/main/assets/img/univasf-logo.png" width=200>
<h3>
    UNIVERSIDADE FEDERAL DO VALE DO SÃO FRANCISCO
    <br>COLEGIADO DE ENGENHARIA DE COMPUTAÇÃO
</h3>

<h3>Orientador</h3>
<span>Prof. Dr. Rosalvo Ferreira de Oliveira Neto</span>

<h3>Discentes</h3>
<span>Anísio Pereira Batista Filho
<br>Edjair Aguiar Gomes Filho
<br>Elayne Rute Lessa Lemos</span>
</center>
<br><br>

## Regressão com Redes Neurais

Projeto em: [https://github.com/anisiobfilho/regression_neuralnetworks_univasf](https://github.com/anisiobfilho/regression_neuralnetworks_univasf)

### Importação das bibliotecas

In [107]:
#OS
import os
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Matplotlib
import matplotlib.pyplot as plt

### Importação das bases de dados

In [108]:
#repo_url = 'https://raw.githubusercontent.com/anisiobfilho/regression_neuralnetworks_univasf/main'

In [109]:
#ic_house_pred_train = pd.read_csv(f'{repo_url}/data/ic_house_pred_train.csv')
ic_house_pred_train = pd.read_csv('data/ic_house_pred_train.csv')
ic_house_pred_train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,136,20,RL,80.0,10400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal,174000
1,1453,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
2,763,60,FV,72.0,8640,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,Con,Normal,215200
3,933,20,RL,84.0,11670,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,320000
4,436,60,RL,43.0,10667,Pave,NaN,IR2,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2009,ConLw,Normal,212000


In [110]:
#ic_house_pred_test = pd.read_csv(f'{repo_url}/data/ic_house_pred_test.csv')
ic_house_pred_test = pd.read_csv('data/ic_house_pred_test.csv')
ic_house_pred_test.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,893,20,RL,70.0,8414,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2006,WD,Normal,154500
1,1106,60,RL,98.0,12256,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,325000
2,414,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,115000
3,523,50,RM,50.0,5000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,159000
4,1037,20,RL,89.0,12898,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,9,2009,WD,Normal,315500


### Eliminar a variável irrelevante PID (Nominal): Parcel identification number

In [111]:
ic_house_pred_train.drop(columns='Id', inplace=True)
ic_house_pred_train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,80.0,10400,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal,174000
1,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
2,60,FV,72.0,8640,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2010,Con,Normal,215200
3,20,RL,84.0,11670,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,320000
4,60,RL,43.0,10667,Pave,NaN,IR2,Lvl,AllPub,CulDSac,...,0,NaN,NaN,NaN,0,4,2009,ConLw,Normal,212000


In [112]:
ic_house_pred_test.drop(columns='Id', inplace=True)
ic_house_pred_test.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,70.0,8414,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2006,WD,Normal,154500
1,60,RL,98.0,12256,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,325000
2,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,115000
3,50,RM,50.0,5000,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,159000
4,20,RL,89.0,12898,Pave,NaN,IR1,HLS,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2009,WD,Normal,315500


### Tratamento dos Missing Values (Apenas colocar para a média)

#### ic_house_pred_train

In [113]:
ic_house_pred_train.columns[ic_house_pred_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [114]:
ic_house_pred_train['LotFrontage'] = pd.to_numeric(ic_house_pred_train['LotFrontage'], errors='coerce')
ic_house_pred_train['LotFrontage'].fillna(ic_house_pred_train['LotFrontage'].mean(), inplace=True)

Alley_replacemap = {'Grvl':1, 'Pave':2, 'NA':3}
ic_house_pred_train['Alley'].replace(Alley_replacemap, inplace=True)
ic_house_pred_train['Alley'].fillna(round(ic_house_pred_train['Alley'].mean()), inplace=True)

MasVnrType_replacemap = {'BrkCmn':1, 'BrkFace':2, 'CBlock':3, 'None':4, 'Stone':5}
ic_house_pred_train['MasVnrType'].replace(MasVnrType_replacemap, inplace=True)
ic_house_pred_train['MasVnrType'].fillna(round(ic_house_pred_train['MasVnrType'].mean()), inplace=True)

ic_house_pred_train['MasVnrArea'].fillna(ic_house_pred_train['MasVnrArea'].mean(), inplace=True)

BsmtQual_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_train['BsmtQual'].replace(BsmtQual_replacemap, inplace=True)
ic_house_pred_train['BsmtQual'].fillna(round(ic_house_pred_train['BsmtQual'].mean()), inplace=True)

BsmtCond_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_train['BsmtCond'].replace(BsmtCond_replacemap, inplace=True)                                         
ic_house_pred_train['BsmtCond'].fillna(round(ic_house_pred_train['BsmtCond'].mean()), inplace=True)

BsmtExposure_replacemap = {'Gd':1, 'Av':2, 'Mn':3, 'No':4, 'NA':5}
ic_house_pred_train['BsmtExposure'].replace(BsmtExposure_replacemap, inplace=True)                                            
ic_house_pred_train['BsmtExposure'].fillna(round(ic_house_pred_train['BsmtExposure'].mean()), inplace=True)

BsmtFinType1_replacemap = {'GLQ':1, 'ALQ':2, 'BLQ':3, 'Rec':4, 'LwQ':5, 'Unf':6, 'NA':7}
ic_house_pred_train['BsmtFinType1'].replace(BsmtFinType1_replacemap, inplace=True)                                             
ic_house_pred_train['BsmtFinType1'].fillna(round(ic_house_pred_train['BsmtFinType1'].mean()), inplace=True)

BsmtFinType2_replacemap = {'GLQ':1, 'ALQ':2, 'BLQ':3, 'Rec':4, 'LwQ':5, 'Unf':6, 'NA':7}
ic_house_pred_train['BsmtFinType2'].replace(BsmtFinType2_replacemap, inplace=True)                                         
ic_house_pred_train['BsmtFinType2'].fillna(round(ic_house_pred_train['BsmtFinType2'].mean()), inplace=True)

Electrical_replacemap = {'SBrkr':1, 'FuseA':2, 'FuseF':3, 'FuseP':4, 'Mix':5}
ic_house_pred_train['Electrical'].replace(Electrical_replacemap, inplace=True)                                    
ic_house_pred_train['Electrical'].fillna(round(ic_house_pred_train['Electrical'].mean()), inplace=True)

FireplaceQu_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_train['FireplaceQu'].replace(FireplaceQu_replacemap, inplace=True)                                           
ic_house_pred_train['FireplaceQu'].fillna(round(ic_house_pred_train['FireplaceQu'].mean()), inplace=True)

GarageType_replacemap = {'2Types':1, 'Attchd':2, 'Basment':3, 'BuiltIn':4, 'CarPort':5, 'Detchd':6, 'NA':7}
ic_house_pred_train['GarageType'].replace(GarageType_replacemap, inplace=True)                                            
ic_house_pred_train['GarageType'].fillna(round(ic_house_pred_train['GarageType'].mean()), inplace=True)

ic_house_pred_train['GarageYrBlt'].fillna(round(ic_house_pred_train['GarageYrBlt'].mean()), inplace=True)
                                         
GarageFinish_replacemap = {'Fin':1, 'RFn':2, 'Unf':3, 'NA':4}
ic_house_pred_train['GarageFinish'].replace(GarageFinish_replacemap, inplace=True)                                          
ic_house_pred_train['GarageFinish'].fillna(round(ic_house_pred_train['GarageFinish'].mean()), inplace=True)

GarageQual_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_train['GarageQual'].replace(GarageQual_replacemap, inplace=True)                                         
ic_house_pred_train['GarageQual'].fillna(round(ic_house_pred_train['GarageQual'].mean()), inplace=True)

GarageCond_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_train['GarageCond'].replace(GarageCond_replacemap, inplace=True)                                         
ic_house_pred_train['GarageCond'].fillna(round(ic_house_pred_train['GarageCond'].mean()), inplace=True)

PoolQC_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'NA':5}
ic_house_pred_train['PoolQC'].replace(PoolQC_replacemap, inplace=True)                                         
ic_house_pred_train['PoolQC'].fillna(round(ic_house_pred_train['PoolQC'].mean()), inplace=True)

Fence_replacemap = {'GdPrv':1, 'MnPrv':2, 'GdWo':3, 'MnWw':4, 'NA':5}
ic_house_pred_train['Fence'].replace(Fence_replacemap, inplace=True)                                        
ic_house_pred_train['Fence'].fillna(round(ic_house_pred_train['Fence'].mean()), inplace=True)

MiscFeature_replacemap = {'Elev':1, 'Gar2':2, 'Othr':3, 'Shed':4, 'TenC':5, 'NA':6,}
ic_house_pred_train['MiscFeature'].replace(MiscFeature_replacemap, inplace=True)                                           
ic_house_pred_train['MiscFeature'].fillna(round(ic_house_pred_train['MiscFeature'].mean()), inplace=True)

In [115]:
ic_house_pred_train.columns[ic_house_pred_train.isnull().any()]

Index([], dtype='object')

In [116]:
ic_house_pred_train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,80.0,10400,Pave,1.0,Reg,Lvl,AllPub,Inside,...,0,2.0,2.0,4.0,0,5,2008,WD,Normal,174000
1,180,RM,35.0,3675,Pave,1.0,Reg,Lvl,AllPub,Inside,...,0,2.0,2.0,4.0,0,5,2006,WD,Normal,145000
2,60,FV,72.0,8640,Pave,1.0,Reg,Lvl,AllPub,Inside,...,0,2.0,2.0,4.0,0,6,2010,Con,Normal,215200
3,20,RL,84.0,11670,Pave,1.0,IR1,Lvl,AllPub,Corner,...,0,2.0,2.0,4.0,0,3,2007,WD,Normal,320000
4,60,RL,43.0,10667,Pave,1.0,IR2,Lvl,AllPub,CulDSac,...,0,2.0,2.0,4.0,0,4,2009,ConLw,Normal,212000


In [117]:
Alley_reversemap = {1: 'Grvl', 2: 'Pave', 3: 'NA'}
ic_house_pred_train['Alley'].replace(Alley_reversemap, inplace=True)

MasVnrType_reversemap = {1: 'BrkCmn', 2: 'BrkFace', 3: 'CBlock', 4: 'None', 5: 'Stone'}
ic_house_pred_train['MasVnrType'].replace(MasVnrType_reversemap, inplace=True)

BsmtQual_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_train['BsmtQual'].replace(BsmtQual_reversemap, inplace=True)

BsmtCond_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_train['BsmtCond'].replace(BsmtCond_reversemap, inplace=True)                                         

BsmtExposure_reversemap = {1: 'Gd', 2: 'Av', 3: 'Mn', 4: 'No', 5: 'NA'}
ic_house_pred_train['BsmtExposure'].replace(BsmtExposure_reversemap, inplace=True)                                            

BsmtFinType1_reversemap = {1: 'GLQ', 2: 'ALQ', 3: 'BLQ', 4: 'Rec', 5: 'LwQ', 6: 'Unf', 7: 'NA'}
ic_house_pred_train['BsmtFinType1'].replace(BsmtFinType1_reversemap, inplace=True)                                             

BsmtFinType2_reversemap = {1: 'GLQ', 2: 'ALQ', 3: 'BLQ', 4: 'Rec', 5: 'LwQ', 6: 'Unf', 7: 'NA'}
ic_house_pred_train['BsmtFinType2'].replace(BsmtFinType2_reversemap, inplace=True)                                         

Electrical_reversemap = {1: 'SBrkr', 2: 'FuseA', 3: 'FuseF', 4: 'FuseP', 5: 'Mix'}
ic_house_pred_train['Electrical'].replace(Electrical_reversemap, inplace=True)                                    

FireplaceQu_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_train['FireplaceQu'].replace(FireplaceQu_reversemap, inplace=True)                                           

GarageType_reversemap = {1: '2Types', 2: 'Attchd', 3: 'Basment', 4: 'BuiltIn', 5: 'CarPort', 6: 'Detchd', 7: 'NA'}
ic_house_pred_train['GarageType'].replace(GarageType_reversemap, inplace=True)                                            
                                         
GarageFinish_reversemap = {1: 'Fin', 2: 'RFn', 3: 'Unf', 4: 'NA'}
ic_house_pred_train['GarageFinish'].replace(GarageFinish_reversemap, inplace=True)                                          

GarageQual_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_train['GarageQual'].replace(GarageQual_reversemap, inplace=True)                                         

GarageCond_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_train['GarageCond'].replace(GarageCond_reversemap, inplace=True)                                         

PoolQC_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'NA'}
ic_house_pred_train['PoolQC'].replace(PoolQC_reversemap, inplace=True)                                         

Fence_reversemap = {1: 'GdPrv', 2: 'MnPrv', 3: 'GdWo', 4: 'MnWw', 5: 'NA'}
ic_house_pred_train['Fence'].replace(Fence_reversemap, inplace=True)                                        

MiscFeature_reversemap = {1: 'Elev', 2: 'Gar2', 3: 'Othr', 4: 'Shed', 5: 'TenC', 6: 'NA'}
ic_house_pred_train['MiscFeature'].replace(MiscFeature_reversemap, inplace=True)       

#print(dict([(value, key) for key, value in old_dict.items()]))

In [118]:
ic_house_pred_train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,80.0,10400,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,5,2008,WD,Normal,174000
1,180,RM,35.0,3675,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,5,2006,WD,Normal,145000
2,60,FV,72.0,8640,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,6,2010,Con,Normal,215200
3,20,RL,84.0,11670,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0,Gd,MnPrv,Shed,0,3,2007,WD,Normal,320000
4,60,RL,43.0,10667,Pave,Grvl,IR2,Lvl,AllPub,CulDSac,...,0,Gd,MnPrv,Shed,0,4,2009,ConLw,Normal,212000


#### ic_house_pred_test

In [119]:
ic_house_pred_test.columns[ic_house_pred_test.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [120]:
ic_house_pred_test['LotFrontage'] = pd.to_numeric(ic_house_pred_test['LotFrontage'], errors='coerce')
ic_house_pred_test['LotFrontage'].fillna(ic_house_pred_test['LotFrontage'].mean(), inplace=True)

Alley_replacemap = {'Grvl':1, 'Pave':2, 'NA':3}
ic_house_pred_test['Alley'].replace(Alley_replacemap, inplace=True)
ic_house_pred_test['Alley'].fillna(round(ic_house_pred_test['Alley'].mean()), inplace=True)

MasVnrType_replacemap = {'BrkCmn':1, 'BrkFace':2, 'CBlock':3, 'None':4, 'Stone':5}
ic_house_pred_test['MasVnrType'].replace(MasVnrType_replacemap, inplace=True)
ic_house_pred_test['MasVnrType'].fillna(round(ic_house_pred_test['MasVnrType'].mean()), inplace=True)

ic_house_pred_test['MasVnrArea'].fillna(ic_house_pred_test['MasVnrArea'].mean(), inplace=True)

BsmtQual_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_test['BsmtQual'].replace(BsmtQual_replacemap, inplace=True)
ic_house_pred_test['BsmtQual'].fillna(round(ic_house_pred_test['BsmtQual'].mean()), inplace=True)

BsmtCond_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_test['BsmtCond'].replace(BsmtCond_replacemap, inplace=True)                                         
ic_house_pred_test['BsmtCond'].fillna(round(ic_house_pred_test['BsmtCond'].mean()), inplace=True)

BsmtExposure_replacemap = {'Gd':1, 'Av':2, 'Mn':3, 'No':4, 'NA':5}
ic_house_pred_test['BsmtExposure'].replace(BsmtExposure_replacemap, inplace=True)                                            
ic_house_pred_test['BsmtExposure'].fillna(round(ic_house_pred_test['BsmtExposure'].mean()), inplace=True)

BsmtFinType1_replacemap = {'GLQ':1, 'ALQ':2, 'BLQ':3, 'Rec':4, 'LwQ':5, 'Unf':6, 'NA':7}
ic_house_pred_test['BsmtFinType1'].replace(BsmtFinType1_replacemap, inplace=True)                                             
ic_house_pred_test['BsmtFinType1'].fillna(round(ic_house_pred_test['BsmtFinType1'].mean()), inplace=True)

BsmtFinType2_replacemap = {'GLQ':1, 'ALQ':2, 'BLQ':3, 'Rec':4, 'LwQ':5, 'Unf':6, 'NA':7}
ic_house_pred_test['BsmtFinType2'].replace(BsmtFinType2_replacemap, inplace=True)                                         
ic_house_pred_test['BsmtFinType2'].fillna(round(ic_house_pred_test['BsmtFinType2'].mean()), inplace=True)

#Electrical_replacemap = {'SBrkr':1, 'FuseA':2, 'FuseF':3, 'FuseP':4, 'Mix':5}
#ic_house_pred_test['Electrical'].replace(Electrical_replacemap, inplace=True)                                    
#ic_house_pred_test['Electrical'].fillna(round(ic_house_pred_test['Electrical'].mean()), inplace=True)

FireplaceQu_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_test['FireplaceQu'].replace(FireplaceQu_replacemap, inplace=True)                                           
ic_house_pred_test['FireplaceQu'].fillna(round(ic_house_pred_test['FireplaceQu'].mean()), inplace=True)

GarageType_replacemap = {'2Types':1, 'Attchd':2, 'Basment':3, 'BuiltIn':4, 'CarPort':5, 'Detchd':6, 'NA':7}
ic_house_pred_test['GarageType'].replace(GarageType_replacemap, inplace=True)                                            
ic_house_pred_test['GarageType'].fillna(round(ic_house_pred_test['GarageType'].mean()), inplace=True)

ic_house_pred_test['GarageYrBlt'].fillna(round(ic_house_pred_test['GarageYrBlt'].mean()), inplace=True)
                                         
GarageFinish_replacemap = {'Fin':1, 'RFn':2, 'Unf':3, 'NA':4}
ic_house_pred_test['GarageFinish'].replace(GarageFinish_replacemap, inplace=True)                                          
ic_house_pred_test['GarageFinish'].fillna(round(ic_house_pred_test['GarageFinish'].mean()), inplace=True)

GarageQual_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_test['GarageQual'].replace(GarageQual_replacemap, inplace=True)                                         
ic_house_pred_test['GarageQual'].fillna(round(ic_house_pred_test['GarageQual'].mean()), inplace=True)

GarageCond_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'NA':6}
ic_house_pred_test['GarageCond'].replace(GarageCond_replacemap, inplace=True)                                         
ic_house_pred_test['GarageCond'].fillna(round(ic_house_pred_test['GarageCond'].mean()), inplace=True)

PoolQC_replacemap = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'NA':5}
ic_house_pred_test['PoolQC'].replace(PoolQC_replacemap, inplace=True)                                         
ic_house_pred_test['PoolQC'].fillna(round(ic_house_pred_test['PoolQC'].mean()), inplace=True)

Fence_replacemap = {'GdPrv':1, 'MnPrv':2, 'GdWo':3, 'MnWw':4, 'NA':5}
ic_house_pred_test['Fence'].replace(Fence_replacemap, inplace=True)                                        
ic_house_pred_test['Fence'].fillna(round(ic_house_pred_test['Fence'].mean()), inplace=True)

MiscFeature_replacemap = {'Elev':1, 'Gar2':2, 'Othr':3, 'Shed':4, 'TenC':5, 'NA':6,}
ic_house_pred_test['MiscFeature'].replace(MiscFeature_replacemap, inplace=True)                                           
ic_house_pred_test['MiscFeature'].fillna(round(ic_house_pred_test['MiscFeature'].mean()), inplace=True)

In [121]:
ic_house_pred_test.columns[ic_house_pred_test.isnull().any()]

Index([], dtype='object')

In [122]:
ic_house_pred_test.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,70.0,8414,Pave,2.0,Reg,Lvl,AllPub,Inside,...,0,3.0,2.0,4.0,0,2,2006,WD,Normal,154500
1,60,RL,98.0,12256,Pave,2.0,IR1,Lvl,AllPub,Corner,...,0,3.0,2.0,4.0,0,4,2010,WD,Normal,325000
2,30,RM,56.0,8960,Pave,1.0,Reg,Lvl,AllPub,Inside,...,0,3.0,2.0,4.0,0,3,2010,WD,Normal,115000
3,50,RM,50.0,5000,Pave,2.0,Reg,Lvl,AllPub,Corner,...,0,3.0,2.0,4.0,0,10,2006,WD,Normal,159000
4,20,RL,89.0,12898,Pave,2.0,IR1,HLS,AllPub,Inside,...,0,3.0,2.0,4.0,0,9,2009,WD,Normal,315500


In [123]:
Alley_reversemap = {1: 'Grvl', 2: 'Pave', 3: 'NA'}
ic_house_pred_test['Alley'].replace(Alley_reversemap, inplace=True)

MasVnrType_reversemap = {1: 'BrkCmn', 2: 'BrkFace', 3: 'CBlock', 4: 'None', 5: 'Stone'}
ic_house_pred_test['MasVnrType'].replace(MasVnrType_reversemap, inplace=True)

BsmtQual_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_test['BsmtQual'].replace(BsmtQual_reversemap, inplace=True)

BsmtCond_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_test['BsmtCond'].replace(BsmtCond_reversemap, inplace=True)                                         

BsmtExposure_reversemap = {1: 'Gd', 2: 'Av', 3: 'Mn', 4: 'No', 5: 'NA'}
ic_house_pred_test['BsmtExposure'].replace(BsmtExposure_reversemap, inplace=True)                                            

BsmtFinType1_reversemap = {1: 'GLQ', 2: 'ALQ', 3: 'BLQ', 4: 'Rec', 5: 'LwQ', 6: 'Unf', 7: 'NA'}
ic_house_pred_test['BsmtFinType1'].replace(BsmtFinType1_reversemap, inplace=True)                                             

BsmtFinType2_reversemap = {1: 'GLQ', 2: 'ALQ', 3: 'BLQ', 4: 'Rec', 5: 'LwQ', 6: 'Unf', 7: 'NA'}
ic_house_pred_test['BsmtFinType2'].replace(BsmtFinType2_reversemap, inplace=True)                                         

#Electrical_reversemap = {1: 'SBrkr', 2: 'FuseA', 3: 'FuseF', 4: 'FuseP', 5: 'Mix'}
#ic_house_pred_test['Electrical'].replace(Electrical_reversemap, inplace=True)                                    

FireplaceQu_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_test['FireplaceQu'].replace(FireplaceQu_reversemap, inplace=True)                                           

GarageType_reversemap = {1: '2Types', 2: 'Attchd', 3: 'Basment', 4: 'BuiltIn', 5: 'CarPort', 6: 'Detchd', 7: 'NA'}
ic_house_pred_test['GarageType'].replace(GarageType_reversemap, inplace=True)                                            
                                         
GarageFinish_reversemap = {1: 'Fin', 2: 'RFn', 3: 'Unf', 4: 'NA'}
ic_house_pred_test['GarageFinish'].replace(GarageFinish_reversemap, inplace=True)                                          

GarageQual_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_test['GarageQual'].replace(GarageQual_reversemap, inplace=True)                                         

GarageCond_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'Po', 6: 'NA'}
ic_house_pred_test['GarageCond'].replace(GarageCond_reversemap, inplace=True)                                         

PoolQC_reversemap = {1: 'Ex', 2: 'Gd', 3: 'TA', 4: 'Fa', 5: 'NA'}
ic_house_pred_test['PoolQC'].replace(PoolQC_reversemap, inplace=True)                                         

Fence_reversemap = {1: 'GdPrv', 2: 'MnPrv', 3: 'GdWo', 4: 'MnWw', 5: 'NA'}
ic_house_pred_test['Fence'].replace(Fence_reversemap, inplace=True)                                        

MiscFeature_reversemap = {1: 'Elev', 2: 'Gar2', 3: 'Othr', 4: 'Shed', 5: 'TenC', 6: 'NA'}
ic_house_pred_test['MiscFeature'].replace(MiscFeature_reversemap, inplace=True)       

#print(dict([(value, key) for key, value in old_dict.items()]))

In [124]:
ic_house_pred_test.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,70.0,8414,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,TA,MnPrv,Shed,0,2,2006,WD,Normal,154500
1,60,RL,98.0,12256,Pave,Pave,IR1,Lvl,AllPub,Corner,...,0,TA,MnPrv,Shed,0,4,2010,WD,Normal,325000
2,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,TA,MnPrv,Shed,0,3,2010,WD,Normal,115000
3,50,RM,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,Corner,...,0,TA,MnPrv,Shed,0,10,2006,WD,Normal,159000
4,20,RL,89.0,12898,Pave,Pave,IR1,HLS,AllPub,Inside,...,0,TA,MnPrv,Shed,0,9,2009,WD,Normal,315500


### Transformação da variável alvo com a função logaritmo natural np.log(x)

In [125]:
ic_house_pred_train['SalePrice_log'] = np.log(ic_house_pred_train['SalePrice'])
ic_house_pred_train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SalePrice_log
0,20,RL,80.0,10400,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Gd,MnPrv,Shed,0,5,2008,WD,Normal,174000,12.066811
1,180,RM,35.0,3675,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Gd,MnPrv,Shed,0,5,2006,WD,Normal,145000,11.884489
2,60,FV,72.0,8640,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Gd,MnPrv,Shed,0,6,2010,Con,Normal,215200,12.279323
3,20,RL,84.0,11670,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,Gd,MnPrv,Shed,0,3,2007,WD,Normal,320000,12.676076
4,60,RL,43.0,10667,Pave,Grvl,IR2,Lvl,AllPub,CulDSac,...,Gd,MnPrv,Shed,0,4,2009,ConLw,Normal,212000,12.264342


In [126]:
ic_house_pred_test['SalePrice_log'] = np.log(ic_house_pred_test['SalePrice'])
ic_house_pred_test.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SalePrice_log
0,20,RL,70.0,8414,Pave,Pave,Reg,Lvl,AllPub,Inside,...,TA,MnPrv,Shed,0,2,2006,WD,Normal,154500,11.947949
1,60,RL,98.0,12256,Pave,Pave,IR1,Lvl,AllPub,Corner,...,TA,MnPrv,Shed,0,4,2010,WD,Normal,325000,12.691580
2,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,TA,MnPrv,Shed,0,3,2010,WD,Normal,115000,11.652687
3,50,RM,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,Corner,...,TA,MnPrv,Shed,0,10,2006,WD,Normal,159000,11.976659
4,20,RL,89.0,12898,Pave,Pave,IR1,HLS,AllPub,Inside,...,TA,MnPrv,Shed,0,9,2009,WD,Normal,315500,12.661914


### Exportar duas bases de dados:

#### A: Missing para a média e variável alvo original

In [127]:
path = 'data/preprocessed'
if not os.path.exists(path):
    os.mkdir(path)

ic_house_pred_train_mean_orig = ic_house_pred_train.drop(columns='SalePrice_log')
ic_house_pred_train_mean_orig.to_csv(f'{path}/ic_house_pred_train_mean_orig.csv', index=False)

ic_house_pred_test_mean_orig = ic_house_pred_test.drop(columns='SalePrice_log')
ic_house_pred_test_mean_orig.to_csv(f'{path}/ic_house_pred_test_mean_orig.csv', index=False)

#### B: Missing para a média e variável alvo transformada pela função logaritmo natural

In [128]:
path = 'data/preprocessed'
if not os.path.exists(path):
    os.mkdir(path)

ic_house_pred_train_mean_log = ic_house_pred_train.drop(columns='SalePrice')
ic_house_pred_train_mean_log.to_csv(f'{path}/ic_house_pred_train_mean_log.csv', index=False)

ic_house_pred_test_mean_log = ic_house_pred_test.drop(columns='SalePrice')
ic_house_pred_test_mean_log.to_csv(f'{path}/ic_house_pred_test_mean_log.csv', index=False)